### Dementia Prediction

Using MRI data from patients w/ and w/o Alzheimier's, develop model that can predict if a patient is demented

Based on: https://www.kaggle.com/ruslankl/dementia-prediction-w-tree-based-models

In [192]:
import pandas as pd
import numpy as np
from functools import reduce
import seaborn as sns

# Import data to dataframe
cs_df = pd.read_csv('Projects/DementiaPrediction/data/oasis_cross-sectional.csv', index_col='ID')
long_df = pd.read_csv('Projects/DementiaPrediction/data/oasis_longitudinal.csv', index_col=['Subject ID', 'MRI ID'])
set_columns_to_category_dtype(cs_df)
set_columns_to_category_dtype(long_df)

In [193]:
cs_df.head()

,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
ID,,,,,,,,,,,
OAS1_0001_MR1,F,R,74,2.0,3.0,29.0,0.0,1344,0.743,1.306,NaN
OAS1_0002_MR1,F,R,55,4.0,1.0,29.0,0.0,1147,0.810,1.531,NaN
OAS1_0003_MR1,F,R,73,4.0,3.0,27.0,0.5,1454,0.708,1.207,NaN
OAS1_0004_MR1,M,R,28,NaN,NaN,NaN,NaN,1588,0.803,1.105,NaN
OAS1_0005_MR1,M,R,18,NaN,NaN,NaN,NaN,1737,0.848,1.010,NaN


In [194]:
long_df.head()

<div>
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th>Group</th>
      <th>Visit</th>
      <th>MR Delay</th>
      <th>M/F</th>
      <th>Hand</th>
      <th>Age</th>
      <th>EDUC</th>
      <th>SES</th>
      <th>MMSE</th>
      <th>CDR</th>
      <th>eTIV</th>
      <th>nWBV</th>
      <th>ASF</th>
    </tr>
    <tr>
      <th>Subject ID</th>
      <th>MRI ID</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="2" valign="top">OAS2_0001</th>
      <th>OAS2_0001_MR1</th>
      <td>Nondemented</td>
      <td>1</td>
      <td>0</td>
      <td>M</td>
      <td>R</td>
      <td>87</td>
      <td>14</td>
      <td>2.0</td>
      <td>27.0</td>
      <td>0.0</td>
      <td>1987</td>
      <td>0.696</td>
      <td>0.883</td>
    </tr>
    <tr>
      <th>OAS2_0001_MR2</th>
      <td>Nondemented</td>
      <td>2</td>
      <td>457</td>
      <td>M</td>
      <td>R</td>
      <td>88</td>
      <td>14</td>
      <td>2.0</td>
      <td>30.0</td>
      <td>0.0</td>
      <td>2004</td>
      <td>0.681</td>
      <td>0.876</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">OAS2_0002</th>
      <th>OAS2_0002_MR1</th>
      <td>Demented</td>
      <td>1</td>
      <td>0</td>
      <td>M</td>
      <td>R</td>
      <td>75</td>
      <td>12</td>
      <td>NaN</td>
      <td>23.0</td>
      <td>0.5</td>
      <td>1678</td>
      <td>0.736</td>
      <td>1.046</td>
    </tr>
    <tr>
      <th>OAS2_0002_MR2</th>
      <td>Demented</td>
      <td>2</td>
      <td>560</td>
      <td>M</td>
      <td>R</td>
      <td>76</td>
      <td>12</td>
      <td>NaN</td>
      <td>28.0</td>
      <td>0.5</td>
      <td>1738</td>
      <td>0.713</td>
      <td>1.010</td>
    </tr>
    <tr>
      <th>OAS2_0002_MR3</th>
      <td>Demented</td>
      <td>3</td>
      <td>1895</td>
      <td>M</td>
      <td>R</td>
      <td>80</td>
      <td>12</td>
      <td>NaN</td>
      <td>22.0</td>
      <td>0.5</td>
      <td>1698</td>
      <td>0.701</td>
      <td>1.034</td>
    </tr>
  </tbody>
</table>
</div>

In [195]:
cs_df.describe()

,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
count,436.000000,235.000000,216.000000,235.00000,235.000000,436.000000,436.000000,436.000000,20.00000
mean,51.357798,3.178723,2.490741,27.06383,0.285106,1481.919725,0.791670,1.198894,20.55000
std,25.269862,1.311510,1.120593,3.69687,0.383405,158.740866,0.059937,0.128682,23.86249
min,18.000000,1.000000,1.000000,14.00000,0.000000,1123.000000,0.644000,0.881000,1.00000
25%,23.000000,2.000000,2.000000,26.00000,0.000000,1367.750000,0.742750,1.111750,2.75000
50%,54.000000,3.000000,2.000000,29.00000,0.000000,1475.500000,0.809000,1.190000,11.00000
75%,74.000000,4.000000,3.000000,30.00000,0.500000,1579.250000,0.842000,1.284250,30.75000
max,96.000000,5.000000,5.000000,30.00000,2.000000,1992.000000,0.893000,1.563000,89.00000


In [196]:
long_df.describe()

,Visit,MR Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,373.000000,373.000000,373.000000,373.000000,354.000000,371.000000,373.000000,373.000000,373.000000,373.000000
mean,1.882038,595.104558,77.013405,14.597855,2.460452,27.342318,0.290885,1488.128686,0.729568,1.195461
std,0.922843,635.485118,7.640957,2.876339,1.134005,3.683244,0.374557,176.139286,0.037135,0.138092
min,1.000000,0.000000,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,0.644000,0.876000
25%,1.000000,0.000000,71.000000,12.000000,2.000000,27.000000,0.000000,1357.000000,0.700000,1.099000
50%,2.000000,552.000000,77.000000,15.000000,2.000000,29.000000,0.000000,1470.000000,0.729000,1.194000
75%,2.000000,873.000000,82.000000,16.000000,3.000000,30.000000,0.500000,1597.000000,0.756000,1.293000
max,5.000000,2639.000000,98.000000,23.000000,5.000000,30.000000,2.000000,2004.000000,0.837000,1.587000


In [293]:
from functools import reduce
def get_data_quality_report(df: pd.DataFrame):
    """
    Create data quality report for both continuous and categorical features

    :param df: dataframe to analyze
    :return: data quality report for continuous features, data quality report for categorical features
    :rtype: pd.Dataframe, pd.Dataframe
    """

    '''
    COMMON DQR CHARACTERISTICS
    '''
    # Total number of rows in dataset
    num_rows = df.shape[0]

    # Number of records as nulls
    nulls = df.isnull().sum()
    nulls.name = 'nulls'

    # Percent of records as nulls
    nulls_pct = nulls / num_rows
    nulls_pct.name = 'nulls pct'

    # Number of unique values in each column
    cardinality = df.nunique()
    cardinality.name = 'cardinality'

    '''
    CREATE DQR FOR NUMERIC/CONTINUOUS FEATURES
    '''
    # Baseline describe function (filters out non-numeric fields)
    continuous_dqr = df.describe()
    continuous_cols = list(continuous_dqr.columns)

    # Modify or add supplemental rows
    continuous_dqr.loc['count'] = num_rows
    continuous_dqr = continuous_dqr.append(nulls[continuous_cols])
    continuous_dqr = continuous_dqr.append(nulls_pct[continuous_cols])
    continuous_dqr = continuous_dqr.append(cardinality[continuous_cols])

    '''
    CREATE DQR FOR CATEGORICAL FEATURES
    '''
    categorical_cols = [col_name for col_name in df.dtypes.where(df.dtypes == 'category').dropna().index if col_name != df.index.name]
    categorical_df = df[categorical_cols]

    # Calculate mode data for each column and aggregate results into single dataframe
    categorical_mode_list = []
    for col in categorical_df:
        categorical_mode_list.append(get_mode_and_second_mode(categorical_df[col]))
    categorical_mode = reduce(lambda x, y: x.merge(y, left_index=True, right_index=True), categorical_mode_list)

    # Aggregate all categorical data quality rows
    categorical_dqr = pd.DataFrame()
    categorical_dqr = categorical_dqr.append(pd.Series([num_rows for col in categorical_df.columns], name='count', index=categorical_df.columns))
    categorical_dqr = categorical_dqr.append(categorical_mode)
    categorical_dqr = categorical_dqr.append(nulls[categorical_cols])
    categorical_dqr = categorical_dqr.append(nulls_pct[categorical_cols])
    categorical_dqr = categorical_dqr.append(cardinality[categorical_cols])

    '''
    Apply formatting
    '''
    def two_decimal_precision(x):
        return "%.2F" % x
    continuous_dqr.loc['nulls pct'] = continuous_dqr.loc['nulls pct'].apply(two_decimal_precision)
    categorical_dqr.loc['nulls pct'] = categorical_dqr.loc['nulls pct'].apply(two_decimal_precision)
    categorical_dqr.loc['mode pct'] = categorical_dqr.loc['mode pct'].apply(two_decimal_precision)
    categorical_dqr.loc['2nd mode pct'] = categorical_dqr.loc['2nd mode pct'].apply(two_decimal_precision)

    '''
    Identify columns that were not listed as continuous or categorical
    '''
    error_cols = list(set(df.columns).difference(set(continuous_cols + categorical_cols)))

    return continuous_dqr, categorical_dqr, error_cols


def get_mode_and_second_mode(s: pd.Series):
    """
    Returns mode, second mode if available, and counts/percentage for both
    :param s: pd.Series to analyze
    :return: pd.DataFrame containing mode and 2nd mode information
    """
    counts = s.value_counts()
    mode_col = counts.index[0]
    mode_counts = counts.iloc[0]
    mode_pct = mode_counts / s.size

    if counts.index.size < 2:
        second_mode_col = np.NaN
        second_mode_counts = np.NaN
        second_mode_pct = np.NaN
    else:
        second_mode_col = counts.index[1]
        second_mode_counts = counts.iloc[1]
        second_mode_pct = second_mode_counts / s.size
    mode_df = pd.DataFrame(
        {
            s.name: [mode_col, mode_counts, mode_pct, second_mode_col, second_mode_counts, second_mode_pct]
        },
        index=['mode', 'mode count', 'mode pct', '2nd mode', '2nd mode count', '2nd mode pct']
    )

    return mode_df


def set_columns_to_category_dtype(df: pd.DataFrame, cols=None):
    # if no columns are passed in, infer categorical columns
    if cols is None:
        cols_to_change = [col_name for col_name in df.dtypes.where(df.dtypes == 'object').dropna().index if col_name != df.index.name]
        for col in cols_to_change:
            df[col] = df[col].astype('category')
    else:
        for col in cols:
            df[col] = df[col].astype('category')

In [199]:

cs_df.head()

,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
ID,,,,,,,,,,,
OAS1_0001_MR1,F,R,74,2.0,3.0,29.0,0.0,1344,0.743,1.306,NaN
OAS1_0002_MR1,F,R,55,4.0,1.0,29.0,0.0,1147,0.810,1.531,NaN
OAS1_0003_MR1,F,R,73,4.0,3.0,27.0,0.5,1454,0.708,1.207,NaN
OAS1_0004_MR1,M,R,28,NaN,NaN,NaN,NaN,1588,0.803,1.105,NaN
OAS1_0005_MR1,M,R,18,NaN,NaN,NaN,NaN,1737,0.848,1.010,NaN


In [295]:
continuous, categorical, error = get_data_quality_report(cs_df)
continuous

,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
count,436,436,436,436,436,436,436,436,436
mean,51.3578,3.17872,2.49074,27.0638,0.285106,1481.92,0.79167,1.19889,20.55
std,25.2699,1.31151,1.12059,3.69687,0.383405,158.741,0.059937,0.128682,23.8625
min,18,1,1,14,0,1123,0.644,0.881,1
25%,23,2,2,26,0,1367.75,0.74275,1.11175,2.75
50%,54,3,2,29,0,1475.5,0.809,1.19,11
75%,74,4,3,30,0.5,1579.25,0.842,1.28425,30.75
max,96,5,5,30,2,1992,0.893,1.563,89
nulls,0,201,220,201,201,0,0,0,416
nulls pct,0.00,0.46,0.50,0.46,0.46,0.00,0.00,0.00,0.95


In [287]:
def two_decimal_precision(x):
    return "%.2F" % x
categorical.loc[['mode pct','nulls pct']].apply(two_decimal_precision)

TypeError: ("cannot convert the series to <class 'float'>", 'occurred at index Hand')